This notebook reproduces the N400 ERP result on the pre-computed Frank et al. 2015 epoched data.

In [1]:
import scipy.io
import pandas as pd
import seaborn as sns

from pathlib import Path
from functools import reduce

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-cjcn3ikb because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
data = scipy.io.loadmat("/om/data/public/language-eeg/frank2015/stimuli_erp.mat", simplify_cells=True)

In [3]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'ERP', 'artefact', 'reject', 'ERPbase', 'dH_pos', 'dH', 'sentences', 'surp_rnn', 'surp_psg', 'surp_pos_psg', 'surp_pos_rnn', 'wordlength', 'surp_ngram', 'surp_ngramfull', 'surp_pos_ngram', 'logwordfreq', 'sentence_position', 'presentation_order', 'SOAs'])

## Prepare data

### N400 results and annotations

In [4]:
data_components = ["ELAN", "LAN", "N400", "EPNP", "P600", "PNP"]
n400_idx = data_components.index("N400")

In [5]:
def make_signal_df(key, target_key=None):
    target_key = target_key or key
    return pd.concat(
        {sentence_idx: pd.DataFrame(mat[:, :, n400_idx], index=pd.RangeIndex(mat.shape[0], name="word_idx")) \
            .reset_index().melt(id_vars=["word_idx"], var_name="subject_idx", value_name=target_key)
        for sentence_idx, mat in enumerate(data[key])},
        names=["sentence_idx", "idx"]) \
        .reset_index().drop(columns=["idx"]).set_index(["subject_idx", "sentence_idx", "word_idx"])

In [6]:
n400_df = pd.merge(make_signal_df("ERP", "value_N400"), make_signal_df("ERPbase", "base_N400"),
                   left_index=True, right_index=True)
n400_df

value_N400  base_N400
subject_idx sentence_idx word_idx                       
0           0            0          -4.587560  -4.639654
                         1          -1.859916  -2.745968
                         2          -0.326976  -0.486989
                         3           3.984934   1.054487
                         4           6.883699   2.721698
...                                       ...        ...
23          204          7          -2.024615  -4.434225
                         8           4.551344   1.269528
                         9           4.102198  -2.200850
                         10         -2.131469   5.741549
                         11         -2.946562  -0.460462

[46344 rows x 2 columns]

In [17]:
n400_df.loc[0].value_N400 - n400_df.loc[0].base_N400

sentence_idx  word_idx
0             0           0.052094
              1           0.886052
              2           0.160013
              3           2.930447
              4           4.162001
                            ...   
204           7          -3.425484
              8           2.878632
              9          -2.466105
              10         -5.354229
              11          6.054846
Length: 1931, dtype: float64

In [7]:
def make_feature_df(key, target_key=None, final_axis_name="subject_idx"):
    target_key = target_key or key
    return pd.concat(
        {sentence_idx: pd.DataFrame(mat, index=pd.RangeIndex(mat.shape[0], name="word_idx")) \
                         .reset_index().melt(id_vars=["word_idx"], var_name=final_axis_name, value_name=target_key)
         for sentence_idx, mat in enumerate(data[key])},
        names=["sentence_idx", "idx"]) \
        .reset_index().drop(columns=["idx"]).set_index(["subject_idx", "sentence_idx", "word_idx"])

In [8]:
by_participant_features = ["artefact", "reject"]
results_df = reduce(lambda acc, feature: pd.merge(acc, make_feature_df(feature), left_index=True, right_index=True),
                    by_participant_features, n400_df)
results_df

value_N400  base_N400  artefact  reject
subject_idx sentence_idx word_idx                                         
0           0            0          -4.587560  -4.639654         0       1
                         1          -1.859916  -2.745968         0       1
                         2          -0.326976  -0.486989         0       0
                         3           3.984934   1.054487         0       0
                         4           6.883699   2.721698         0       1
...                                       ...        ...       ...     ...
23          204          7          -2.024615  -4.434225         0       0
                         8           4.551344   1.269528         0       0
                         9           4.102198  -2.200850         0       0
                         10         -2.131469   5.741549         0       0
                         11         -2.946562  -0.460462         0       1

[46344 rows x 4 columns]

### Stimulus features

In [9]:
def make_surprisal_df(key, target_key=None, i_offset=0):
    target_key = target_key or key
    return pd.concat(
        {sentence_idx: pd.DataFrame(mat, index=pd.RangeIndex(mat.shape[0], name="word_idx"),
                                    columns=[f"{target_key}_{i_offset+idx}" for idx in range(mat.shape[1])])
         for sentence_idx, mat in enumerate(data[key])},
        names=["sentence_idx", "word_idx"])

In [10]:
surprisal_df = pd.concat([make_surprisal_df("surp_ngram", "surp_ngram_order", i_offset=2),
                          make_surprisal_df("surp_rnn", "surp_rnn_size", i_offset=1)],
                         axis=1)
surprisal_df

surp_ngram_order_2  surp_ngram_order_3  \
sentence_idx word_idx                                           
0            0                   3.044363            3.044340   
             1                   5.447571            5.298179   
             2                   4.487278            3.561708   
             3                   1.639671            1.415385   
             4                   7.144369            4.562181   
...                                   ...                 ...   
204          7                   3.851212            4.149281   
             8                   4.030146            4.339636   
             9                   6.416153            6.987241   
             10                  3.648262            2.545669   
             11                  5.210036            1.392226   

                       surp_ngram_order_4  surp_rnn_size_1  surp_rnn_size_2  \
sentence_idx word_idx                                                         
0            0                   3.044340         3.482672         3.145689   
             1                   5.292745         9.514440         8.214565   
             2                   3.084290         9.111075         8.330564   
             3                   0.676790         3.594287         3.210452   
             4                   4.629370         8.459698         8.541803   
...                                   ...              ...              ...   
204          7                   4.245483         4.631127         3.924689   
             8                   4.272032         8.139806         7.945024   
             9                   6.910058        10.800107         9.959173   
             10                  2.342811         4.568191         3.884360   
             11                  1.327797         9.073960         8.574935   

                       surp_rnn_size_3  surp_rnn_size_4  surp_rnn_size_5  \
sentence_idx word_idx                                                      
0            0                3.121044         2.511534         2.865847   
             1                7.030273         6.441496         5.929814   
             2                6.914139         5.580521         5.000933   
             3                2.787990         2.840195         1.975472   
             4                8.242652         8.169230         8.290066   
...                                ...              ...              ...   
204          7                3.901125         3.368505         3.373968   
             8                6.722867         6.237772         5.792343   
             9                8.835783         8.886982         8.520737   
             10               3.874658         3.318967         3.432938   
             11               8.116138         8.222916         7.337699   

                       surp_rnn_size_6  surp_rnn_size_7  surp_rnn_size_8  \
sentence_idx word_idx                                                      
0            0                2.992646         3.094798         2.977805   
             1                5.246744         5.131305         5.886331   
             2                4.906635         4.620777         4.347193   
             3                1.707842         1.536145         1.337063   
             4                8.272437         7.855889         7.984851   
...                                ...              ...              ...   
204          7                3.190540         2.908144         2.281913   
             8                5.422487         4.335986         3.894010   
             9                8.073103         8.324292         6.935927   
             10               3.411648         3.273675         2.664433   
             11               6.672644         5.709465         5.870685   

                       surp_rnn_size_9  surp_rnn_size_10  
sentence_idx word_idx                                     
0            0                3.289495          2.993

In [11]:
def make_control_df(key):
    return pd.concat([pd.DataFrame({key: wf}, index=pd.RangeIndex(len(wf), name="word_idx"))
                      for idx, wf in enumerate(data[key])],
                     names=["sentence_idx"], keys=list(range(len(data[key]))))

In [12]:
control_features = ["logwordfreq", "wordlength"]
control_df = reduce(lambda acc, feature: pd.merge(acc, make_control_df(feature), left_index=True, right_index=True),
                    control_features[1:], make_control_df(control_features[0]))
control_df

logwordfreq  wordlength
sentence_idx word_idx                         
0            0           -5.250051           1
             1           -8.384545           6
             2           -8.332967           4
             3           -5.541296           3
             4           -6.162017           5
...                            ...         ...
204          7           -5.442780           3
             8           -7.989015           4
             9           -8.380188           6
             10          -5.442780           3
             11          -7.027121           5

[1931 rows x 2 columns]

## Merge and analyze

In [13]:
merged_df = pd.merge(results_df, surprisal_df, left_index=True, right_index=True)
merged_df = pd.merge(merged_df, control_df, left_index=True, right_index=True)
assert len(merged_df) == len(n400_df)

In [14]:
merged_df

value_N400  base_N400  artefact  reject  \
sentence_idx word_idx subject_idx                                            
0            0        0             -4.587560  -4.639654         0       1   
                      1              0.888646   3.267604         0       1   
                      2              2.465251   0.202261         0       1   
                      3              0.476780  -2.252108         0       1   
                      4             -3.044633   1.923643         0       1   
...                                       ...        ...       ...     ...   
204          11       19             4.360778  -3.634742         0       1   
                      20            -3.397018  -3.558512         0       1   
                      21            -3.438822   7.059365         0       1   
                      22            10.746570  -3.774443         0       1   
                      23            -2.946562  -0.460462         0       1   

                                   surp_ngram_order_2  surp_ngram_order_3  \
sentence_idx word_idx subject_idx                                           
0            0        0                      3.044363            3.044340   
                      1                      3.044363            3.044340   
                      2                      3.044363            3.044340   
                      3                      3.044363            3.044340   
                      4                      3.044363            3.044340   
...                                               ...                 ...   
204          11       19                     5.210036            1.392226   
                      20                     5.210036            1.392226   
                      21                     5.210036            1.392226   
                      22                     5.210036            1.392226   
                      23                     5.210036            1.392226   

                                   surp_ngram_order_4  surp_rnn_size_1  \
sentence_idx word_idx subject_idx                                        
0            0        0                      3.044340         3.482672   
                      1                      3.044340         3.482672   
                      2                      3.044340         3.482672   
                      3                      3.044340         3.482672   
                      4                      3.044340         3.482672   
...                                               ...              ...   
204          11       19                     1.327797         9.073960   
                      20                     1.327797         9.073960   
                      21                     1.327797         9.073960   
                      22                     1.327797         9.073960   
                      23                     1.327797         9.073960   

                                   surp_rnn_size_2  surp_rnn_size_3  \
sentence_idx word_idx subject_idx                                     
0            0        0                   3.145689         3.121044   
                      1                   3.145689         3.121044   
                      2                   3.145689         3.121044   
                      3                   3.145689         3.121044   
                      4                   3.145689         3.121044   
...                                            ...              ...   
204          11       19                  8.574935         8.116138   
                      20                  8.574935         8.116138   
                      21                  8.574935         8.116138   
                      22                  8.574935         8.116138   
                      23                  8.574935         8.116138   

                                   surp_rnn_size_4  surp_rnn_size_5  \
sentence_idx word_idx subject_idx                           

In [15]:
merged_df.to_csv("frank_erp_n400.csv")